In [1]:
using BenchmarkTools
using Combinatorics

include("../phase1/node.jl")
include("../phase1/edges.jl")
include("../phase1/graph.jl")
include("../phase1/read_stsp.jl")
include("../phase2/comp_connexes.jl")
include("../phase1/main.jl")
include("../phase2/queue.jl")
include("../phase2/heuristics.jl")
include("../phase2/Kruskal.jl")
include("../phase2/prims_algorithm.jl")

prims_algorithm

In [2]:
tree_root = TreeNode("1", 0)
child1 = TreeNode("2", 0)
child2 = TreeNode("3", 0)
child3 = TreeNode("4", 0)
child4 = TreeNode("5",0)
child5 = TreeNode("6",0)
child6 = TreeNode("7",0)

tree = Tree("test tree", Vector{TreeNode{Int}}())

first_row = [ child1, child2, child3]
second_row = [child4, child6]
add_node!(tree, tree_root)
for child in first_row
    add_node!(tree, child, parent = tree_root)
end
for child in second_row
    add_node!(tree, child, parent = child3)
end
add_node!(tree, child5, parent = child4)
show(tree)

Node 1 has  rank 0 and 3 children.
It has no parent.
Its children are: 
     2
     3
     4
Node 2 has  rank 0 and 0 children.
Its parent is 1
It has no children.
Node 3 has  rank 0 and 0 children.
Its parent is 1
It has no children.
Node 4 has  rank 0 and 2 children.
Its parent is 1
Its children are: 
     5
     7
Node 5 has  rank 0 and 1 children.
Its parent is 4
Its children are: 
     6
Node 7 has  rank 0 and 0 children.
Its parent is 4
It has no children.
Node 6 has  rank 0 and 0 children.
Its parent is 5
It has no children.


In [3]:
n1= TreeNode("1", 0)
n2 = TreeNode("2", 0)
n3 = TreeNode("3", 0)
n4= TreeNode("4", 0)
t1 = Tree("test tree" , Vector{TreeNode{Int}}())
t1 = add_node!(t1, n1)
t1 = add_node!(t1, n2, parent = n1)
t1 = add_node!(t1, n3)
t1 = add_node!(t1, n4, parent = n3)
rank_union!(t1, n1, n3)


(TreeNode{Int64}("1", 0, nothing, [2, 3], 0, 1, 1), TreeNode{Int64}("3", 0, 1, [4], 0, 0, 3))

In [4]:
show(t1)

Node 1 has  rank 1 and 2 children.
It has no parent.
Its children are: 
     2
     3
Node 2 has  rank 0 and 0 children.
Its parent is 1
It has no children.
Node 3 has  rank 0 and 1 children.
Its parent is 1
Its children are: 
     4
Node 4 has  rank 0 and 0 children.
Its parent is 3
It has no children.


In [5]:
"""crée une liste d'adjacence a partir d'un graphe"""
function adjacency_dict( graph::Graph) 
    #Dictionaire de adjacence
    adj_dict = Dict()
    #Dictionaire de correspondance entre les noeuds et les items de la file de priorite
    correspondance_dict = Dict()
    #Chaque noeud est une clef du dictionaire de adjacence
    for (i, node) in enumerate(nodes(graph))
        adj_dict[i] = Dict()
        correspondance_dict[name(node)] = i
    end
    #Ajoute les voisins de chaque noeud dans le dictionaire de adjacence
    for edge in edges(graph)
        node1, node2 = nodes(edge)
        idx1 = correspondance_dict[name(node1)]
        idx2 = correspondance_dict[name(node2)]
        adj_dict[idx1][idx2] = weight(edge)
        adj_dict[idx2][idx1] = weight(edge)
    end
    return adj_dict
end

"""Crée une file de priorite des arbres à partir d'un graphe. Le noeud de depart a une priorite de 0 et les autres ont une priorite de Inf"""
function prims_priority_queue(graph::Graph{Y,T}, start_node_name::String) where {Y,T}
    priority_queue = PriorityQueue{PriorityItem{TreeNode{Y}}}()
    tree = Tree(name(graph), Vector{TreeNode{Y}}())
    for node in nodes(graph)
        if name(node) == start_node_name
            blank_node = TreeNode(name(node), data(node))
            priority_item = PriorityItem( 0 , blank_node)
        else
            blank_node = TreeNode(name(node), data(node))
            priority_item = PriorityItem( Inf, blank_node)
        end
        add_node!(tree, blank_node)
        push!(priority_queue, priority_item)
    end
    return priority_queue, tree
end

"""Implementation de l'algorithme de Prim"""
function prims_algorithm(graph::Graph{Y,T}; start_node_name::Any = nothing) where {Y,T}
    #initialisation
    if isnothing(start_node_name)
        start_node_name = name(nodes(graph)[1])
    end
    #initialisation de la file de priorite et du dictionaire d'adjacence
    priority_queue, tree = prims_priority_queue(graph, start_node_name)
    adjacency_list = adjacency_dict(graph)
    #sauvegarde de la racine
    root = poplast!(priority_queue)
    priority_node = root
    #boucle principale
    while !is_empty(priority_queue)
        for  (neighbor_idx, edge_weight) in adjacency_list[index(data(priority_node))]
            #Met a jour la priorite du voisin si elle est dans le priority_que et elle est plus petite que la priorite actuelle
            for item in priority_queue.items
                #println("nodes(tree)[neighbor_idx] = ", nodes(tree)[neighbor_idx])
                if index(data(item)) == neighbor_idx
                    if  edge_weight < priority(item)
                        change_parent!(tree, nodes(tree)[neighbor_idx], data(priority_node))
                        change_dist!( nodes(tree)[neighbor_idx], edge_weight)
                        priority!(item, edge_weight)
                    end
                break
                end
            end
        end
        #prend le noeud avec la plus petite priorite
        priority_node = poplast!(priority_queue)         
    end
    return tree, data(root)
end

prims_algorithm

In [6]:
graphe_test = Graph("Test",Node{Vector{Float64}}[],Edge{Int,Vector{Float64}}[])

#Nodes 
nodea = Node("a",[0.])
nodeb = Node("b",[0.])
nodec = Node("c",[0.])
noded = Node("d",[0.])
nodee = Node("e",[0.])
nodef = Node("f",[0.])
nodeg = Node("g",[0.])
nodeh = Node("h",[0.])
nodei = Node("i",[0.])
node_list = [nodea,nodeb,nodec, noded,nodee, nodef, 
                nodeg, nodeh, nodei]

#Edges
edge1 = Edge(nodea,nodeb, 4)
edge2 = Edge(nodea,nodeh, 8)
edge3 = Edge(nodeb, nodeh, 11)
edge4 = Edge(nodeb, nodec, 8)
edge5 = Edge(nodeh, nodei, 7)
edge6 = Edge(nodeh, nodeg, 1)
edge7 = Edge(nodeg, nodei, 6)
edge8 = Edge(nodeg, nodef, 2)
edge9 = Edge(nodec, nodef, 4)
edge10 = Edge(nodei, nodec, 2)
edge11 = Edge(nodef, nodee, 10)
edge12 = Edge(nodee, noded, 9)
edge13 = Edge(nodec,noded, 7)
edge14 = Edge(noded, nodef, 14)
edge_list = [edge1, edge2, edge3, edge4, edge5, edge6,
                edge7, edge8, edge9, edge10, edge11,
                edge12, edge13, edge14]

##Construction du graphe
for i in node_list
    add_node!(graphe_test,i)
end
for i in edge_list
    add_edge!(graphe_test, i)
end


In [7]:
priority_queue, tree = prims_priority_queue(graphe_test, "a")

p_tree, root = prims_algorithm(graphe_test, start_node_name = "a")
p_graph = tree_to_graph(p_tree, root)

kruskal_tree, root = kruskal(graphe_test, start_node_name = "a")
k_graph = tree_to_graph(kruskal_tree, root)
sum_of_weights(k_graph)


37.0

In [8]:
#@benchmark kruskal(graphe_test)

In [9]:
@time p_tree1, root1 = prims_algorithm(graphe_test, start_node_name = "a")

  0.000067 seconds (385 allocations: 25.266 KiB)


(Tree{Vector{Float64}}("Test", TreeNode{Vector{Float64}}[TreeNode{Vector{Float64}}("a", [0.0], nothing, [2], 0, 0, 1), TreeNode{Vector{Float64}}("b", [0.0], 1, [3], 4, 0, 2), TreeNode{Vector{Float64}}("c", [0.0], 2, [4, 6, 9], 8, 0, 3), TreeNode{Vector{Float64}}("d", [0.0], 3, [5], 7, 0, 4), TreeNode{Vector{Float64}}("e", [0.0], 4, Int64[], 9, 0, 5), TreeNode{Vector{Float64}}("f", [0.0], 3, [7], 4, 0, 6), TreeNode{Vector{Float64}}("g", [0.0], 6, [8], 2, 0, 7), TreeNode{Vector{Float64}}("h", [0.0], 7, Int64[], 1, 0, 8), TreeNode{Vector{Float64}}("i", [0.0], 3, Int64[], 2, 0, 9)]), TreeNode{Vector{Float64}}("a", [0.0], nothing, [2], 0, 0, 1))

In [10]:
swiss_42_graph, swiss_42_nodes = graph_from_tsp("../../instances/stsp/swiss42.tsp","graphe1")
# println("running Prim's algorithm on swiss42")
# @time swiss_42_tree, root2 = prims_algorithm(swiss_42_graph)
# swiss_42 = tree_to_graph(swiss_42_tree, root2)
# sum_of_weights(swiss_42)

Reading of header : ✓
Reading of nodes : ✓
Reading of edges : ✓


(Graph{Vector{Float64}, Float64}("graphe1", Node{Vector{Float64}}[Node{Vector{Float64}}("1", [0.8414709848078965, 0.5403023058681398]), Node{Vector{Float64}}("2", [1.8185948536513634, -0.8322936730942848]), Node{Vector{Float64}}("3", [0.4233600241796016, -2.9699774898013365]), Node{Vector{Float64}}("4", [-1.5136049906158564, -1.3072872417272239]), Node{Vector{Float64}}("5", [-0.9589242746631385, 0.28366218546322625]), Node{Vector{Float64}}("6", [-0.8382464945967776, 2.880510859951098]), Node{Vector{Float64}}("7", [1.3139731974375781, 1.5078045086866092]), Node{Vector{Float64}}("8", [1.9787164932467636, -0.2910000676172271]), Node{Vector{Float64}}("9", [0.8242369704835132, -1.8222605237693539]), Node{Vector{Float64}}("10", [-1.6320633326681093, -2.517214587229357])  …  Node{Vector{Float64}}("33", [2.9997355803218015, -0.03983024166917844]), Node{Vector{Float64}}("34", [0.5290826861200238, -0.8485702747846052]), Node{Vector{Float64}}("35", [-0.428182669496151, -0.9036922050915067]), Node

In [11]:
show(swiss_42_graph)

Graph graphe1 has 42 nodes.
Node 1, data: 

[0.8414709848078965, 0.5403023058681398]
Node 2, data: [1.8185948536513634, -0.8322936730942848]
Node 3, data: [0.4233600241796016, -2.9699774898013365]
Node 4, data: [-1.5136049906158564, -1.3072872417272239]
Node 5, data: [-0.9589242746631385, 0.28366218546322625]
Node 6, data: [-0.8382464945967776, 2.880510859951098]
Node 7, data: [1.3139731974375781, 1.5078045086866092]
Node 8, data: [1.9787164932467636, -0.2910000676172271]
Node 9, data: [0.8242369704835132, -1.8222605237693539]
Node 10, data: [-1.6320633326681093, -2.517214587229357]
Node 11, data: [-0.9999902065507035, 0.004425697988050785]
Node 12, data: [-0.5365729180004349, 0.8438539587324921]
Node 13, data: [1.2605011104799226, 2.7223403443505885]
Node 14, data: [0.9906073556948704, 0.1367372182078336]
Node 15, data: [0.6502878401571168, -0.7596879128588213]
Node 16, data: [-0.2879033166650653, -0.9576594803233847]
Node 17, data: [-1.9227949837591136, -0.5503266761031939]
Node 18, data: [-0.7509872467716762, 0.66031670824408

7 à 8, poids: 45.0
Arête de 7 à 9, poids: 85.0
Arête de 7 à 10, poids: 111.0
Arête de 7 à 11, poids: 75.0
Arête de 7 à 12, poids: 82.0
Arête de 7 à 13, poids: 69.0
Arête de 7 à 14, poids: 60.0
Arête de 7 à 15, poids: 63.0
Arête de 7 à 16, poids: 70.0
Arête de 7 à 17, poids: 71.0
Arête de 7 à 18, poids: 85.0
Arête de 7 à 19, poids: 44.0
Arête de 7 à 20, poids: 52.0
Arête de 7 à 21, poids: 115.0
Arête de 7 à 22, poids: 161.0
Arête de 7 à 23, poids: 136.0
Arête de 7 à 24, poids: 122.0
Arête de 7 à 25, poids: 210.0
Arête de 7 à 26, poids: 91.0
Arête de 7 à 27, poids: 25.0
Arête de 7 à 28, poids: 37.0
Arête de 7 à 29, poids: 54.0
Arête de 7 à 30, poids: 78.0
Arête de 7 à 31, poids: 81.0
Arête de 7 à 32, poids: 90.0
Arête de 7 à 33, poids: 68.0
Arête de 7 à 34, poids: 136.0
Arête de 7 à 35, poids: 116.0
Arête de 7 à 36, poids: 150.0
Arête de 7 à 37, poids: 147.0
Arête de 7 à 38, poids: 76.0
Arête de 7 à 39, poids: 148.0
Arête de 7 à 40, poids: 147.0
Arête de 7 à 41, poids: 180.0
Arête de 7 à

 à 42, poids: 135.0
Arête de 15 à 1, poids: 74.0
Arête de 15 à 2, poids: 63.0
Arête de 15 à 3, poids: 95.0
Arête de 15 à 4, poids: 84.0
Arête de 15 à 5, poids: 83.0
Arête de 15 à 6, poids: 56.0
Arête de 15 à 7, poids: 63.0
Arête de 15 à 8, poids: 42.0
Arête de 15 à 9, poids: 148.0
Arête de 15 à 10, poids: 174.0
Arête de 15 à 11, poids: 134.0
Arête de 15 à 12, poids: 129.0
Arête de 15 à 13, poids: 117.0
Arête de 15 à 14, poids: 59.0
Arête de 15 à 16, poids: 11.0
Arête de 15 à 17, poids: 8.0
Arête de 15 à 18, poids: 63.0
Arête de 15 à 19, poids: 93.0
Arête de 15 à 20, poids: 35.0
Arête de 15 à 21, poids: 135.0
Arête de 15 à 22, poids: 223.0
Arête de 15 à 23, poids: 195.0
Arête de 15 à 24, poids: 184.0
Arête de 15 à 25, poids: 273.0
Arête de 15 à 26, poids: 146.0
Arête de 15 à 27, poids: 71.0
Arête de 15 à 28, poids: 95.0
Arête de 15 à 29, poids: 113.0
Arête de 15 à 30, poids: 138.0
Arête de 15 à 31, poids: 138.0
Arête de 15 à 32, poids: 81.0
Arête de 15 à 33, poids: 107.0
Arête de 15 à 3

 à 27, poids: 23.0
Arête de 19 à 28, poids: 60.0
Arête de 19 à 29, poids: 70.0
Arête de 19 à 30, poids: 81.0
Arête de 19 à 31, poids: 95.0
Arête de 19 à 32, poids: 134.0
Arête de 19 à 33, poids: 101.0
Arête de 19 à 34, poids: 172.0
Arête de 19 à 35, poids: 149.0
Arête de 19 à 36, poids: 194.0
Arête de 19 à 37, poids: 190.0
Arête de 19 à 38, poids: 115.0
Arête de 19 à 39, poids: 160.0
Arête de 19 à 40, poids: 138.0
Arête de 19 à 41, poids: 159.0
Arête de 19 à 42, poids: 80.0
Arête de 20 à 1, poids: 78.0
Arête de 20 à 2, poids: 63.0
Arête de 20 à 3, poids: 88.0
Arête de 20 à 4, poids: 78.0
Arête de 20 à 5, poids: 72.0
Arête de 20 à 6, poids: 34.0
Arête de 20 à 7, poids: 52.0
Arête de 20 à 8, poids: 60.0
Arête de 20 à 9, poids: 129.0
Arête de 20 à 10, poids: 157.0
Arête de 20 à 11, poids: 110.0
Arête de 20 à 12, poids: 98.0
Arête de 20 à 13, poids: 88.0
Arête de 20 à 14, poids: 25.0
Arête de 20 à 15, poids: 35.0
Arête de 20 à 16, poids: 46.0
Arête de 20 à 17, poids: 39.0
Arête de 20 à 18,

 à 14, poids: 44.0
Arête de 27 à 15, poids: 71.0
Arête de 27 à 16, poids: 80.0
Arête de 27 à 17, poids: 78.0
Arête de 27 à 18, poids: 106.0
Arête de 27 à 19, poids: 23.0
Arête de 27 à 20, poids: 48.0
Arête de 27 à 21, poids: 139.0
Arête de 27 à 22, poids: 160.0
Arête de 27 à 23, poids: 145.0
Arête de 27 à 24, poids: 115.0
Arête de 27 à 25, poids: 211.0
Arête de 27 à 26, poids: 75.0
Arête de 27 à 28, poids: 53.0
Arête de 27 à 29, poids: 68.0
Arête de 27 à 30, poids: 86.0
Arête de 27 à 31, poids: 95.0
Arête de 27 à 32, poids: 114.0
Arête de 27 à 33, poids: 90.0
Arête de 27 à 34, poids: 160.0
Arête de 27 à 35, poids: 139.0
Arête de 27 à 36, poids: 173.0
Arête de 27 à 37, poids: 168.0
Arête de 27 à 38, poids: 92.0
Arête de 27 à 39, poids: 162.0
Arête de 27 à 40, poids: 150.0
Arête de 27 à 41, poids: 176.0
Arête de 27 à 42, poids: 101.0
Arête de 28 à 1, poids: 28.0
Arête de 28 à 2, poids: 33.0
Arête de 28 à 3, poids: 4.0
Arête de 28 à 4, poids: 11.0
Arête de 28 à 5, poids: 19.0
Arête de 28 

93.0
Arête de 35 à 4, poids: 96.0
Arête de 35 à 5, poids: 107.0
Arête de 35 à 6, poids: 139.0
Arête de 35 à 7, poids: 116.0
Arête de 35 à 8, poids: 104.0
Arête de 35 à 9, poids: 135.0
Arête de 35 à 10, poids: 142.0
Arête de 35 à 11, poids: 151.0
Arête de 35 à 12, poids: 181.0
Arête de 35 à 13, poids: 168.0
Arête de 35 à 14, poids: 174.0
Arête de 35 à 15, poids: 146.0
Arête de 35 à 16, poids: 143.0
Arête de 35 à 17, poids: 152.0
Arête de 35 à 18, poids: 100.0
Arête de 35 à 19, poids: 149.0
Arête de 35 à 20, poids: 159.0
Arête de 35 à 21, poids: 23.0
Arête de 35 à 22, poids: 168.0
Arête de 35 à 23, poids: 106.0
Arête de 35 à 24, poids: 168.0
Arête de 35 à 25, poids: 200.0
Arête de 35 à 26, poids: 173.0
Arête de 35 à 27, poids: 139.0
Arête de 35 à 28, poids: 89.0
Arête de 35 à 29, poids: 85.0
Arête de 35 à 30, poids: 96.0
Arête de 35 à 31, poids: 75.0
Arête de 35 à 32, poids: 81.0
Arête de 35 à 33, poids: 49.0
Arête de 35 à 34, poids: 24.0
Arête de 35 à 36, poids: 104.0
Arête de 35 à 37, 


Arête de 42 à 32, poids: 190.0
Arête de 42 à 33, poids: 132.0
Arête de 42 à 34, poids: 194.0
Arête de 42 à 35, poids: 170.0
Arête de 42 à 36, poids: 246.0
Arête de 42 à 37, poids: 253.0
Arête de 42 à 38, poids: 187.0
Arête de 42 à 39, poids: 137.0
Arête de 42 à 40, poids: 80.0
Arête de 42 à 41, poids: 81.0


In [12]:
#running kruskal on swiss 42
println("running Kruskal's algorithm on swiss42")
swiss_42_tree, root = kruskal(swiss_42_graph)
swiss_42 = tree_to_graph(swiss_42_tree, root)
sum_of_weights(swiss_42)

running Kruskal's algorithm on swiss42


1079.0

In [13]:
graphe1, graphe1_nodes = graph_from_tsp("../../instances/stsp/bays29.tsp","graphe1")
println("running Prim's algorithm")
@time bays_29_tree, root3 = prims_algorithm(graphe1)
bays_29_graph = tree_to_graph(bays_29_tree, root3)

sum_of_weights(bays_29_graph)

Reading of header : ✓
Reading of nodes : ✓
Reading of edges : ✓
running Prim's algorithm
  0.032165 seconds (39.63 k allocations: 2.426 MiB, 97.66% compilation time)


1557.0

In [14]:
println("running Kruskal's algorithm")
bays_29, bays_29_tree,root = kruskal(graphe1)
#show(bays_29_tree)
#bays_29_graph = tree_to_graph(bays_29_tree, root)
sum_of_weights(bays_29_graph)

running Kruskal's algorithm


BoundsError: BoundsError: attempt to access Tuple{Tree{Vector{Float64}}, TreeNode{Vector{Float64}}} at index [3]

## Algorithm de Held et Karp 

In [15]:
# nodea = Node("a",[0.])
# nodeb = Node("b",[0.])
# nodec = Node("c",[0.])
# noded = Node("d",[0.])
# nodee = Node("e",[0.])
# node_list = [nodea,nodeb,nodec, noded,nodee]
# #Edges are fully connected
# edge1 = Edge(nodea,nodeb, 4.)
# edge2 = Edge(nodea,nodec, 8.)
# edge3 = Edge(nodea, noded, 11.)
# edge4 = Edge(nodea, nodee, 8.)
# edge5 = Edge(nodeb, nodec, 7.)
# edge6 = Edge(nodeb, noded, 1.)
# edge7 = Edge(nodeb, nodee, 6.)
# edge8 = Edge(nodec, noded, 2.)
# edge9 = Edge(nodec, nodee, 4.)
# edge10 = Edge(noded, nodee, 7.)
# edge_list = [edge1, edge2, edge3, edge4, edge5, edge6,
#                 edge7, edge8, edge9, edge10]

# tsp_test = Graph("Test",node_list,edge_list)
# tsp_test

nodea = Node("a",[0.])
nodeb = Node("b",[0.])
nodec = Node("c",[0.])
noded = Node("d",[0.])
nodee = Node("e",[0.])
node_list = [nodea,nodeb,nodec, noded,nodee]
#Edges are fully connected
edge1 = Edge(nodea,nodeb, 3.)
edge2 = Edge(nodea,nodec, 2.)
edge3 = Edge(nodea, noded, 11.)
edge4 = Edge(nodea, nodee, 9.)
edge5 = Edge(nodeb, nodec, 7.)
edge6 = Edge(nodeb, noded, 12.)
edge7 = Edge(nodeb, nodee, 6.)
edge8 = Edge(nodec, noded, 6.)
edge9 = Edge(nodec, nodee, 4.)
edge10 = Edge(noded, nodee, 7.)
edge_list = [edge1, edge2, edge3, edge4, edge5, edge6,
                edge7, edge8, edge9, edge10]

tsp_test = Graph("Test",node_list,edge_list)
tsp_test



Graph{Vector{Float64}, Float64}("Test", Node{Vector{Float64}}[Node{Vector{Float64}}("a", [0.0]), Node{Vector{Float64}}("b", [0.0]), Node{Vector{Float64}}("c", [0.0]), Node{Vector{Float64}}("d", [0.0]), Node{Vector{Float64}}("e", [0.0])], Edge{Float64, Vector{Float64}}[Edge{Float64, Vector{Float64}}(Node{Vector{Float64}}("a", [0.0]), Node{Vector{Float64}}("b", [0.0]), 3.0), Edge{Float64, Vector{Float64}}(Node{Vector{Float64}}("a", [0.0]), Node{Vector{Float64}}("c", [0.0]), 2.0), Edge{Float64, Vector{Float64}}(Node{Vector{Float64}}("a", [0.0]), Node{Vector{Float64}}("d", [0.0]), 11.0), Edge{Float64, Vector{Float64}}(Node{Vector{Float64}}("a", [0.0]), Node{Vector{Float64}}("e", [0.0]), 9.0), Edge{Float64, Vector{Float64}}(Node{Vector{Float64}}("b", [0.0]), Node{Vector{Float64}}("c", [0.0]), 7.0), Edge{Float64, Vector{Float64}}(Node{Vector{Float64}}("b", [0.0]), Node{Vector{Float64}}("d", [0.0]), 12.0), Edge{Float64, Vector{Float64}}(Node{Vector{Float64}}("b", [0.0]), Node{Vector{Float64}}

In [16]:
function h_k_exact_algorithm(g::Graph ; start_node_name::Any = nothing)
    #if no start node name, first node is start node
    if isnothing(start_node_name)
        start_node_name = name(nodes(g)[1])
    end
    #move start node to first position of graph
    for (i, node) in enumerate(nodes(g))
        if name(node) == start_node_name
            g.nodes[1], g.nodes[i] = g.nodes[i], g.nodes[1]
            break
        end
    end
    #create adjacency list
    adjacency_list = adjacency_dict(g)
    distance_dict = Dict()
    for k in 2:length(nodes(g))
        distance_dict[(Set([k]), k)] = Dict("c" =>adjacency_list[1][k], "p"=> [1, k])
    end
    for s in 2:length(nodes(g))-1
        for subset in combinations(2:length(nodes(g)), s)
            for k in subset
                min_dist = Inf
                for m in subset 
                    if m !=k
                        predecessor = distance_dict[(setdiff(Set(subset), Set([k])), m)]
                        #println("predecessor", predecessor)
                        dist = predecessor["c"] + adjacency_list[m][k]
                        if dist < min_dist
                            min_dist = dist
                            min_m = m
                            new_list = copy(predecessor["p"])
                            append!(new_list, k)
                            distance_dict[(Set(subset), k)] = Dict("c" => min_dist, "p" => new_list)
                        end
                    end
                end
            end
        end
    end

    #returns the minimum path from the start node that goes to all of the other paths
    min = Inf
    final_path = []
    for k in 2:length(nodes(g))
        distance = distance_dict[(Set(2:length(nodes(g))), k)]["c"] + adjacency_list[k][1]
        println("distance = ", distance)
        if distance < min
            println("distance = ", distance)
            min = distance
            final_path = copy(distance_dict[(Set(2:length(nodes(g))), k)]["p"])
            append!(final_path, 1)
        end
    end
    return min, final_path
end


h_k_algorithm(tsp_test)


UndefVarError: UndefVarError: `h_k_algorithm` not defined

In [17]:


function get_shortest_edges(graph::Graph, departure_node::Node)
    shortest_edge_vec = Vector{Edge}(undef, 2)
    shortest_dist_vec = Vector{Float64}([Inf, Inf])
    for  edge in edges(graph)
        node1, node2 = nodes(edge)
        if name(node1) == name(departure_node) || name(node2) == name(departure_node)
            #copies over the previous best to second best
            if weight(edge) < shortest_dist_vec[1]
                if shortest_dist_vec[1]< shortest_dist_vec[2]
                    shortest_dist_vec[2] = shortest_dist_vec[1]
                    shortest_edge_vec[2] = shortest_edge_vec[1]
                end
                shortest_dist_vec[1] = weight(edge)
                shortest_edge_vec[1] = edge
            elseif weight(edge) < shortest_dist_vec[2]
                shortest_dist_vec[2] = weight(edge)
                shortest_edge_vec[2] = edge
            end
        end
    end
    return  shortest_edge_vec[1], shortest_edge_vec[2]
end



function find_one_tree(graph::Graph, departure_node::Node, adjacency_list::Dict)
    #Copies graph to feed into prim's algorithm
    start_graph = deepcopy(graph)
    #removes the departure node from the graph and saves its index
    idx = remove_node!(start_graph, departure_node)
    remove_edges!(start_graph, departure_node)
    #uses prims algorithm to find the one_tree
    one_tree, root = prims_algorithm(start_graph, start_node_name = name(departure_node))
    #converts the prims algorithm to a graph
    one_tree_graph = tree_to_graph(one_tree, root)
    #finds the second shortest edge from the departure node
    shortest_edge_1, shortest_edge_2   = get_shortest_edges(graph, departure_node)
    add_node!(one_tree_graph, departure_node, idx)
    add_edge!(one_tree_graph,shortest_edge_1)
    add_edge!(one_tree_graph,shortest_edge_2)
    one_tree_distance = sum_of_weights(one_tree_graph)
    return one_tree_distance, one_tree_graph
end

#Returns the degrees of nodes in  a graph
function degree(graph::Graph)
    adjacency_list = adjacency_dict(graph)
    degrees = []
    for i in 1:length(nodes(graph))
        push!(degrees, length(keys(adjacency_list[i])))
    end
    return degrees
end

function update_edge_weights!(graph::Graph, pis::Vector{Float64})
    correspondance_dict = Dict()
    for (i, node) in enumerate(nodes(graph))
        correspondance_dict[name(node)] = i
    end
    for edge in edges(graph)
        node1, node2 = nodes(edge)
        node1_idx = correspondance_dict[name(node1)]
        node2_idx = correspondance_dict[name(node2)]
        new_weight = weight(edge) + pis[node1_idx] + pis[node2_idx]
        set_weight!(edge, new_weight)
    end
end

"""subgradient heuristic for calculating a minimal tour"""
function lkh_subgradient(start_graph::Graph;  departure_node::Union{Node, Nothing} = nothing, t_k_method::String = "1/k")
    
    no_nodes = length(nodes(start_graph))
    if isnothing(departure_node)
        departure_node = nodes(start_graph)[1]
    end
    graph = deepcopy(start_graph)
    k = 0
    w = -Inf
    pis = zeros(length(nodes(graph)))
    adjacency_list = adjacency_dict(graph)

    while true && k < 10000000
        iter_time = time()
        total_distance, one_tree = find_one_tree(graph, departure_node, adjacency_list)
        weights_k = total_distance - 2 * sum(pis)
        w = max(w, weights_k)
        v_k = degree(one_tree) .- 2
        #Calculates the l1 norm of v_k
        
        if v_k == zeros(length(nodes(graph)))
            return total_distance, one_tree
        end
        if t_k_method == "1/k"
            t_k = 1/(k+1)
        elseif t_k_method == "blah"
            t_k = 1
        else
            start_weight = sum_of_weights(start_graph)
            t_k = start_weight/(100*no_nodes + k)
        end
        pis = pis + t_k * v_k
        k = k + 1
        update_edge_weights!(graph, pis)
        v_k_norm  = sum(broadcast(abs, v_k))
        println("k = ", k, " time = ", time() - iter_time, " VK_norm ", v_k_norm )
    end
    return Inf, nothing
end

lkh_subgradient

In [18]:
function get_leaves(graph::Graph)
    leaves = []
    adj_dict = adjacency_dict(graph)
    for node in keys(adj_dict)
        if length(keys(adj_dict[node])) == 1
            push!(leaves, node)
        end
    end
    return leaves
end


get_leaves (generic function with 1 method)

In [19]:
p_tree1, root1 = prims_algorithm(tsp_test, start_node_name = "a")
show(p_tree1)
p_graph = tree_to_graph( p_tree1, root1)
get_leaves(p_graph)

Node a has  rank 0 and 2 children.
It has no parent.
Its children are: 
     b
     c
Node b has  rank 0 and 0 children.
Its parent is a
It has no children.
Node c has  rank 0 and 2 children.
Its parent is a
Its children are: 
     e
     d
Node d has  rank 0 and 0 children.
Its parent is c
It has no children.
Node e has  rank 0 and 0 children.
Its parent is c
It has no children.


3-element Vector{Any}:
 5
 4
 2

In [20]:
lkh_subgradient(tsp_test, departure_node = nodec, t_k_method = "node")

k = 1 time = 0.03099513053894043 VK_norm 2
k = 2 time = 0.000102996826171875 VK_norm 2
k = 3 time = 6.29425048828125e-5 VK_norm 2
k = 4 time = 5.91278076171875e-5 VK_norm 2


(24.0, Graph{Vector{Float64}, Float64}("Test", Node{Vector{Float64}}[Node{Vector{Float64}}("a", [0.0]), Node{Vector{Float64}}("b", [0.0]), Node{Vector{Float64}}("c", [0.0]), Node{Vector{Float64}}("d", [0.0]), Node{Vector{Float64}}("e", [0.0])], Edge{Float64, Vector{Float64}}[Edge{Float64, Vector{Float64}}(Node{Vector{Float64}}("a", [0.0]), Node{Vector{Float64}}("b", [0.0]), 3.0), Edge{Float64, Vector{Float64}}(Node{Vector{Float64}}("e", [0.0]), Node{Vector{Float64}}("d", [0.0]), 7.0), Edge{Float64, Vector{Float64}}(Node{Vector{Float64}}("b", [0.0]), Node{Vector{Float64}}("e", [0.0]), 7.337330670935383), Edge{Float64, Vector{Float64}}(Node{Vector{Float64}}("a", [0.0]), Node{Vector{Float64}}("c", [0.0]), 2.0), Edge{Float64, Vector{Float64}}(Node{Vector{Float64}}("c", [0.0]), Node{Vector{Float64}}("d", [0.0]), 4.662669329064617)]))

In [21]:

#Fully connected graph with 7 edges
a = Node("a",[0.])
b = Node("b",[0.])
c = Node("c",[0.])
d = Node("d",[0.])
e = Node("e",[0.])
f = Node("f",[0.])
g = Node("g",[0.])
node_list = [a,b,c, d,e, f, g]
#Edges are fully connected
edge1 = Edge(a,b, 4.)
edge2 = Edge(a,c, 8.)
edge3 = Edge(a, d, 11.)
edge4 = Edge(a, e, 8.)
edge5 = Edge(a, f, 7.)
edge6 = Edge(a, g, 1.)
edge7 = Edge(b, c, 6.)
edge8 = Edge(b, d, 2.)
edge9 = Edge(b, e, 4.)
edge10 = Edge(b, f, 7.)
edge11 = Edge(b, g, 2.)
edge12 = Edge(c, d, 7.)
edge13 = Edge(c, e, 1.)
edge14 = Edge(c, f, 6.)
edge15 = Edge(c, g, 3.)
edge16 = Edge(d, e, 5.)
edge17 = Edge(d, f, 4.)
edge18 = Edge(d, g, 8.)
edge19 = Edge(e, f, 2.)
edge20 = Edge(e, g, 7.)
edge21 = Edge(f, g, 3.)
edge_list = [edge1, edge2, edge3, edge4, edge5, edge6,
                edge7, edge8, edge9, edge10, edge11,
                edge12, edge13, edge14, edge15, edge16,
                edge17, edge18, edge19, edge20, edge21]
#creates the graph
tsp_test2 = Graph("Test2",node_list,edge_list)

# score, test2_graph = lkh_subgradient(tsp_test2, t_k_method = "1/k")
# score

Graph{Vector{Float64}, Float64}("Test2", Node{Vector{Float64}}[Node{Vector{Float64}}("a", [0.0]), Node{Vector{Float64}}("b", [0.0]), Node{Vector{Float64}}("c", [0.0]), Node{Vector{Float64}}("d", [0.0]), Node{Vector{Float64}}("e", [0.0]), Node{Vector{Float64}}("f", [0.0]), Node{Vector{Float64}}("g", [0.0])], Edge{Float64, Vector{Float64}}[Edge{Float64, Vector{Float64}}(Node{Vector{Float64}}("a", [0.0]), Node{Vector{Float64}}("b", [0.0]), 4.0), Edge{Float64, Vector{Float64}}(Node{Vector{Float64}}("a", [0.0]), Node{Vector{Float64}}("c", [0.0]), 8.0), Edge{Float64, Vector{Float64}}(Node{Vector{Float64}}("a", [0.0]), Node{Vector{Float64}}("d", [0.0]), 11.0), Edge{Float64, Vector{Float64}}(Node{Vector{Float64}}("a", [0.0]), Node{Vector{Float64}}("e", [0.0]), 8.0), Edge{Float64, Vector{Float64}}(Node{Vector{Float64}}("a", [0.0]), Node{Vector{Float64}}("f", [0.0]), 7.0), Edge{Float64, Vector{Float64}}(Node{Vector{Float64}}("a", [0.0]), Node{Vector{Float64}}("g", [0.0]), 1.0), Edge{Float64, Vec

In [22]:
gr17_graph, gr17_nodes = graph_from_tsp("../../instances/stsp/gr17.tsp","graphe1")
println("running h_k_algorithm on gr17")
#@time total_distance, one_tree = lkh_subgradient(gr17_graph)

Reading of header : ✓
Reading of nodes : ✓
Reading of edges : ✓
running h_k_algorithm on gr17


In [23]:
"""Brute force calculates TSP solution"""
function brute_force_tsp(g::Graph, start_node::Any)
    adjacency_list = adjacency_dict(g)
    g_nodes = nodes(g)
    nodes_perm = collect(permutations(collect(1:length(g_nodes))))
    min_dist = Inf
    min_perm = []
    for perm in nodes_perm
        dist = 0
        for i in 1:length(perm)-1
            dist += adjacency_list[perm[i]][perm[i+1]]
        end
        dist += adjacency_list[perm[end]][perm[1]]
        println("permutation", perm)
        println("dist = ", dist)
        if dist < min_dist
            min_dist = dist
            min_perm = perm
        end
        
    end
    return min_dist, min_perm
end
brute_force_tsp(tsp_test, "a")

permutation

[1, 2, 3, 4, 5]
dist = 32.0
permutation[1, 2, 3, 5, 4]
dist = 32.0
permutation[1, 2, 4, 3, 5]
dist = 34.0
permutation[1, 2, 4, 5, 3]
dist = 28.0
permutation[1, 2, 5, 3, 4]
dist = 30.0
permutation[1, 2, 5, 4, 3]
dist = 24.0
permutation[1, 3, 2, 4, 5]
dist = 37.0
permutation[1, 3, 2, 5, 4]
dist = 33.0
permutation[1, 3, 4, 2, 5]
dist = 35.0
permutation[1, 3, 4, 5, 2]
dist = 24.0
permutation[1, 3, 5, 2, 4]
dist = 35.0
permutation[1, 3, 5, 4, 2]
dist = 28.0
permutation[1, 4, 2, 3, 5]
dist = 43.0
permutation[1, 4, 2, 5, 3]
dist = 35.0
permutation[1, 4, 3, 2, 5]
dist = 39.0
permutation[1, 4, 3, 5, 2]
dist = 30.0
permutation[1, 4, 5, 2, 3]
dist = 33.0
permutation[1, 4, 5, 3, 2]
dist = 32.0
permutation[1, 5, 2, 3, 4]
dist = 39.0
permutation[1, 5, 2, 4, 3]
dist = 35.0
permutation[1, 5, 3, 2, 4]
dist = 43.0
permutation[1, 5, 3, 4, 2]
dist = 34.0
permutation[1, 5, 4, 2, 3]
dist = 37.0
permutation[1, 5, 4, 3, 2]
dist = 32.0
permutation[2, 1, 3, 4, 5]
dist = 24.0
permutation[2, 1, 3, 5, 4]
dist = 28

(24.0, [1, 2, 5, 4, 3])